# Backtesting Rayner Teos 1689% Bollinger Bands

(Ps: still not sure whether it's the legit way to do the backtesting like this, open for discussion)

![](https://user-images.githubusercontent.com/64836390/261828125-75840c59-0ca9-4284-ae69-c5abb6252d40.jpg)

In [26]:
import vectorbt as vbt
import pandas

In [27]:
data = pandas.read_csv("./BTCUSDT_2021-01-01.csv")
data["datetime"] = pandas.to_datetime(data["datetime"], unit="ms")
data

,datetime,Open,High,Low,Close,Volume
0,2021-01-01 00:00:00,28921.5,29026.0,28703.5,28990.0,917.494
1,2021-01-01 01:00:00,28990.0,29505.5,28956.5,29411.0,1927.830
2,2021-01-01 02:00:00,29411.0,29462.0,29150.0,29218.0,931.537
3,2021-01-01 03:00:00,29218.0,29358.5,29171.5,29303.0,440.535
4,2021-01-01 04:00:00,29303.0,29397.5,29078.0,29229.5,711.549
...,...,...,...,...,...,...
13869,2022-08-01 21:00:00,23117.0,23118.5,22964.5,23012.0,3333.020
13870,2022-08-01 22:00:00,23012.0,23180.0,22900.0,23150.0,3999.116
13871,2022-08-01 23:00:00,23150.0,23399.0,23149.5,23250.0,8659.609
13872,2022-08-02 00:00:00,23250.0,23429.5,23146.5,23176.0,6632.418


In [28]:
data = vbt.YFData.download("BTC-USD", start="2021-01-01",
                           end="2022-08-02"
                           ).concat()
data["Close"]

c:\Users\yongsheng.tan\AppData\Local\miniconda3\envs\quant_lab\lib\site-packages\vectorbt\data\base.py:681: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



Date
2020-12-31 00:00:00+00:00    29001.720703
2021-01-01 00:00:00+00:00    29374.152344
2021-01-02 00:00:00+00:00    32127.267578
2021-01-03 00:00:00+00:00    32782.023438
2021-01-04 00:00:00+00:00    31971.914062
                                 ...     
2022-07-28 00:00:00+00:00    23843.886719
2022-07-29 00:00:00+00:00    23804.632812
2022-07-30 00:00:00+00:00    23656.207031
2022-07-31 00:00:00+00:00    23336.896484
2022-08-01 00:00:00+00:00    23314.199219
Freq: D, Name: BTC-USD, Length: 579, dtype: float64

In [29]:
SMA_200 = vbt.MA.run(data["Close"], window=200)
BBANDS_20 = vbt.talib("BBANDS").run(data["Close"], timeperiod=20,
                           nbdevup=2.5, nbdevdn=2.5)
RSI = vbt.RSI.run(data["Close"], window=2)

In [37]:
out_dict={}
order_price = data["Close"].shift(1)
entries = SMA_200.ma_below(data["Close"]).vbt & BBANDS_20.lowerband_above(data["Close"]).vbt
# generate limit order entries
entries = entries.vbt.signals.generate_ohlc_stop_exits(
    data["Open"], data["High"], data["Low"], data["Close"],
    sl_trail=False,
    out_dict=out_dict,
    sl_stop=0.03, # in %
    exit_wait=1) # buy the trade at next close price when buy signal is triggered

timebased_stop_loss = entries.vbt.signals.fshift(10) # close open trades after holding it for 10 days
exits = (RSI.rsi_crossed_above(50) | timebased_stop_loss)

entries, exits = entries.vbt.signals.clean(exits) # automatically remove ignored exit points

In [38]:
pf= vbt.Portfolio.from_signals(data["Close"],
                          entries,
                          exits,
                          fees=0.002, # in %
                          slippage=0.001, # in %
                          direction="longOnly",
                          init_cash=1000,
                          freq="1d"
                          )
pf.trades.records_readable

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,0,0.020195,2021-12-05 00:00:00+00:00,49418.216504,1.996008,2021-12-06 00:00:00+00:00,50532.042375,2.040996,18.45678,0.018494,Long,Closed,0


In [39]:
pf.stats()

Start                         2020-12-31 00:00:00+00:00
End                           2022-08-01 00:00:00+00:00
Period                                579 days 00:00:00
Start Value                                      1000.0
End Value                                    1018.45678
Total Return [%]                               1.845678
Benchmark Return [%]                         -19.610979
Max Gross Exposure [%]                            100.0
Total Fees Paid                                4.037004
Max Drawdown [%]                               0.299301
Max Drawdown Duration                   1 days 00:00:00
Total Trades                                          1
Total Closed Trades                                   1
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                      100.0
Best Trade [%]                                 1.849369
Worst Trade [%]                                1

In [40]:
pf.plot().show()